In [ ]:
!pip install transformers
!pip install emoji
!pip install contractions

     |████████████████████████████████| 2.5MB 33.3MB/s 
     |████████████████████████████████| 3.3MB 44.0MB/s 
     |████████████████████████████████| 901kB 44.1MB/s 
     |████████████████████████████████| 133kB 27.3MB/s 
     |████████████████████████████████| 327kB 39.0MB/s 
     |████████████████████████████████| 286kB 45.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85404 sha256=d05488bbef60cd9de1d9a52545295ca3630d9931b1533c7c3932025b56ea3476
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
# Data manipulation libraries
import sys, os
import pandas as pd
import numpy as np
import json

import emoji
import contractions
import re

# Scikit-learn packages
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

# Packages to define a BERT model
from transformers import TFBertModel, BertTokenizerFast, BertConfig

# Keras and TensorFlow packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal

In [ ]:
# Importing train, validation and test datasets with preprocessed texts and labels
train_GE = pd.read_excel("/content/drive/MyDrive/train.xlsx")
val_GE = pd.read_excel("/content/drive/MyDrive/val.xlsx")
test_GE = pd.read_excel("/content/drive/MyDrive/test.xlsx")
# Shape validation
print(train_GE.shape)
print(test_GE.shape)

print(val_GE.shape)

(5000, 2)
(501, 2)
(1001, 2)


In [ ]:
print(train_GE.columns)
print(test_GE.columns)

Index(['review', 'sentiment'], dtype='object')
Index(['review', 'sentiment'], dtype='object')


In [ ]:
data = train_GE.copy()
data.drop(["review"],axis=1,inplace=True)
#data["sentiment"].replace(0,"negative",inplace=True)
#data["sentiment"].replace(1,"positive",inplace=True)
data=pd.get_dummies(data)
train = pd.concat([train_GE,data], axis=1, join='inner')
train.drop(["sentiment"],axis=1,inplace=True)

In [ ]:
data1 = val_GE.copy()
data1.drop(["review"],axis=1,inplace=True)
#data1["label"].replace(0,"negative",inplace=True)
#data1["label"].replace(1,"positive",inplace=True)
data1=pd.get_dummies(data1)
val = pd.concat([val_GE,data1], axis=1, join='inner')
val.drop(["sentiment"],axis=1,inplace=True)

In [ ]:
data2 = test_GE.copy()
data2.drop(["review"],axis=1,inplace=True)
#data2.replace(0,"negative",inplace=True)
#data2.replace(1,"positive",inplace=True)
data2=pd.get_dummies(data2)
test = pd.concat([test_GE,data2], axis=1, join='inner')
test.drop(["sentiment"],axis=1,inplace=True)

In [ ]:
full_text = pd.concat([train['review'],test["review"],val["review"]])
max_length = full_text.apply(lambda x: len(x.split())).max()
max_length

1830

In [ ]:
train.columns

Index(['review', 'sentiment_negative', 'sentiment_positive'], dtype='object')

In [ ]:
GE_taxonomy= ['sentiment_negative', 'sentiment_positive']

In [ ]:
train['review'] = train['review'].str.replace(r'[^a-zA-Z ]+', '')
train

,review,sentiment_negative,sentiment_positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production br br The filmin...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically theres a family where a little boy J...,1,0
4,Petter Matteis Love in the Time of Money is a ...,0,1
...,...,...,...
4995,An interesting slasher film with multiple susp...,1,0
4996,i watched this series when it first came out i...,0,1
4997,Once again Jet Li brings his charismatic prese...,0,1
4998,I rented this movie after hearing Chris Gore s...,1,0


In [ ]:
#Importing BERT pre-trained model and tokenizer
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name, output_hidden_states=False)
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# function for creating BERT based model
def create_model(nb_labels):

  # Load the MainLayer
  bert = transformer_model.layers[0]

  # Build the model inputs
  input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
  attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32')
  token_ids = Input(shape=(max_length,), name='token_ids', dtype='int32')
  inputs = {'input_ids': input_ids, 'attention_mask': attention_mask, 'token_ids': token_ids}

  # Load the Transformers BERT model as a layer in a Keras model
  bert_model = bert(inputs)[1]
  dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
  pooled_output = dropout(bert_model, training=False)

  # Then build the model output
  emotion = Dense(units=nb_labels, activation="softmax", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(pooled_output)
  outputs = emotion

  # And combine it all in a model object
  model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel')

  return model

In [ ]:
# Creating a model instance
model = create_model(2)

# Take a look at the model
model.summary

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7f11c63f7350>>

In [ ]:
# Creating train, validation and test variables
X_train = train['review']
y_train = train.loc[:, GE_taxonomy].values.astype(float)

X_val = val['review']
y_val  = val.loc[:, GE_taxonomy].values.astype(float)

X_test = test['review']
y_test = test.loc[:, GE_taxonomy].values.astype(float)

In [ ]:
# Tokenizing train data
train_token = tokenizer(
    text = X_train.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing valisation data
val_token = tokenizer(
    text = X_val.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

# Tokenizing test data
test_token = tokenizer(
    text = X_test.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [ ]:
# Creating BERT compatible inputs with Input Ids, attention masks and token Ids 
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_ids': train_token['token_type_ids']}
val = {'input_ids': val_token['input_ids'], 'attention_mask': val_token['attention_mask'],'token_ids': val_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_ids': test_token['token_type_ids']}

In [ ]:
# Creating TF tensors
train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(32)
val_tensor = tf.data.Dataset.from_tensor_slices((val, y_val)).shuffle(len(val)).batch(32)
test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(32)

In [ ]:
# Function for calculating multilabel class weights
def calculating_class_weights(y_true):
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights

class_weights = calculating_class_weights(y_train)


In [ ]:
# Custom loss function for multilabel
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
filepath = "/content/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath,monitor="val_loss",verbose=1,save_best_only=True,mode="max")
callback_lists = [checkpoint]

In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=3.e-05,
    )

# Set loss
loss = get_weighted_loss(class_weights)

# Compile the model

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

# train the model
history = model.fit(train_tensor, 
                    epochs=25, 
                    validation_data=val_tensor,
                    callbacks = callback_lists)

Epoch 1/25


ResourceExhaustedError: ignored

In [ ]:
# Save model weights
model.save_weights('/content/bert-weights_senti_1.h5')

In [ ]:
y_pred_proba = model.predict(test)

In [ ]:
len = len(y_pred_proba)

In [ ]:
list = []
for i in range(len):
  samples_pred_proba = y_pred_proba[i]
  print(samples_pred_proba)
  max = samples_pred_proba.max()
  result = np.where(samples_pred_proba == max )
  list.append(GE_taxonomy[result[0][0]])
    #print(GE_taxonomy[result])


In [ ]:
test_GE['predictions'] = list

In [ ]:
test_GE.replace(["sentiment_anger","anger"],0,inplace=True)
test_GE.replace(["sentiment_fear","fear"],1,inplace=True)
test_GE.replace(["sentiment_joy","joy"],2,inplace=True)
test_GE.replace(["sentiment_love","love"],3,inplace=True)
test_GE.replace(["sentiment_sadness","sadness"],4,inplace=True)
test_GE.replace(["sentiment_surprise","surprise"],5,inplace=True)

In [ ]:
true = test_GE['sentiment'].tolist()
pred = test_GE['predictions'].tolist()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(true, pred)

In [ ]:
# Retrieving initial preprocessings
def preprocess_corpus(x):
    
    # Adding a space between words and punctation
    x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
    x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)

    # Demojize
    x = emoji.demojize(x)

    # Expand contraction
    x = contractions.fix(x)

    # Lower
    x = x.lower()

    #correct some acronyms/typos/abbreviations  
    x = re.sub(r"lmao", "laughing my ass off", x)  
    x = re.sub(r"amirite", "am i right", x)
    x = re.sub(r"\b(tho)\b", "though", x)
    x = re.sub(r"\b(ikr)\b", "i know right", x)
    x = re.sub(r"\b(ya|u)\b", "you", x)
    x = re.sub(r"\b(eu)\b", "europe", x)
    x = re.sub(r"\b(da)\b", "the", x)
    x = re.sub(r"\b(dat)\b", "that", x)
    x = re.sub(r"\b(dats)\b", "that is", x)
    x = re.sub(r"\b(cuz)\b", "because", x)
    x = re.sub(r"\b(fkn)\b", "fucking", x)
    x = re.sub(r"\b(tbh)\b", "to be honest", x)
    x = re.sub(r"\b(tbf)\b", "to be fair", x)
    x = re.sub(r"faux pas", "mistake", x)
    x = re.sub(r"\b(btw)\b", "by the way", x)
    x = re.sub(r"\b(bs)\b", "bullshit", x)
    x = re.sub(r"\b(kinda)\b", "kind of", x)
    x = re.sub(r"\b(bruh)\b", "bro", x)
    x = re.sub(r"\b(w/e)\b", "whatever", x)
    x = re.sub(r"\b(w/)\b", "with", x)
    x = re.sub(r"\b(w/o)\b", "without", x)
    x = re.sub(r"\b(doj)\b", "department of justice", x)

    # replace some words with multiple occurences of a letter, example "coooool" turns into --> cool
    x = re.sub(r"\b(j+e{2,}z+e*)\b", "jeez", x)
    x = re.sub(r"\b(co+l+)\b", "cool", x)
    x = re.sub(r"\b(g+o+a+l+)\b", "goal", x)
    x = re.sub(r"\b(s+h+i+t+)\b", "shit", x)
    x = re.sub(r"\b(o+m+g+)\b", "omg", x)
    x = re.sub(r"\b(w+t+f+)\b", "wtf", x)
    x = re.sub(r"\b(w+h+a+t+)\b", "what", x)
    x = re.sub(r"\b(y+e+y+|y+a+y+|y+e+a+h+)\b", "yeah", x)
    x = re.sub(r"\b(w+o+w+)\b", "wow", x)
    x = re.sub(r"\b(w+h+y+)\b", "why", x)
    x = re.sub(r"\b(s+o+)\b", "so", x)
    x = re.sub(r"\b(f)\b", "fuck", x)
    x = re.sub(r"\b(w+h+o+p+s+)\b", "whoops", x)
    x = re.sub(r"\b(ofc)\b", "of course", x)
    x = re.sub(r"\b(the us)\b", "usa", x)
    x = re.sub(r"\b(gf)\b", "girlfriend", x)
    x = re.sub(r"\b(hr)\b", "human ressources", x)
    x = re.sub(r"\b(mh)\b", "mental health", x)
    x = re.sub(r"\b(idk)\b", "i do not know", x)
    x = re.sub(r"\b(gotcha)\b", "i got you", x)
    x = re.sub(r"\b(y+e+p+)\b", "yes", x)
    x = re.sub(r"\b(a*ha+h[ha]*|a*ha +h[ha]*)\b", "haha", x)
    x = re.sub(r"\b(o?l+o+l+[ol]*)\b", "lol", x)
    x = re.sub(r"\b(o*ho+h[ho]*|o*ho +h[ho]*)\b", "ohoh", x)
    x = re.sub(r"\b(o+h+)\b", "oh", x)
    x = re.sub(r"\b(a+h+)\b", "ah", x)
    x = re.sub(r"\b(u+h+)\b", "uh", x)

    # Handling emojis
    x = re.sub(r"<3", " love ", x)
    x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
    x = re.sub(r":\)", " smiling_face ", x)
    x = re.sub(r"^_^", " smiling_face ", x)
    x = re.sub(r"\*_\*", " star_struck ", x)
    x = re.sub(r":\(", " frowning_face ", x)
    x = re.sub(r":\^\(", " frowning_face ", x)
    x = re.sub(r";\(", " frowning_face ", x)
    x = re.sub(r":\/",  " confused_face", x)
    x = re.sub(r";\)",  " wink", x)
    x = re.sub(r">__<",  " unamused ", x)
    x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
    x = re.sub(r"\b(n+a+h+)\b", "no", x)
    
    # Handling special cases of text
    x = re.sub(r"h a m b e r d e r s", "hamberders", x)
    x = re.sub(r"b e n", "ben", x)
    x = re.sub(r"s a t i r e", "satire", x)
    x = re.sub(r"y i k e s", "yikes", x)
    x = re.sub(r"s p o i l e r", "spoiler", x)
    x = re.sub(r"thankyou", "thank you", x)
    x = re.sub(r"a^r^o^o^o^o^o^o^o^n^d", "around", x)

    # Remove special characters and numbers replace by space + remove double space
    x = re.sub(r"\b([.]{3,})"," dots ", x)
    x = re.sub(r"[^A-Za-z!?_]+"," ", x)
    x = re.sub(r"\b([s])\b *","", x)
    x = re.sub(r" +"," ", x)
    x = x.strip()

    return x

In [ ]:
def predict_samples(text_samples, model):
    
    # Text preprocessing and cleaning
    text_samples_clean = [preprocess_corpus(text) for text in text_samples]
    
    # Tokenizing train data
    samples_token = tokenizer(
        text = text_samples_clean,
        add_special_tokens = True,
        max_length = max_length,
        truncation = True,
        padding = 'max_length', 
        return_tensors = 'tf',
        return_token_type_ids = True,
        return_attention_mask = True,
        verbose = True,
    )
    
    # Preparing to feed the model
    samples = {'input_ids': samples_token['input_ids'],
               'attention_mask': samples_token['attention_mask'],
               'token_ids': samples_token['token_type_ids']
              }
    
    # Probability predictions
    samples_pred_proba = model.predict(samples)
    print(samples_pred_proba)
    for arr in samples_pred_proba:
      max = arr[0]
      for i in range(1, 2):
        if arr[i] > max:
            max = arr[i]
    # Label prediction using threshold
    samples_pred_labels = proba_to_labels(samples_pred_proba,max)
    #print(samples_pred_labels)       
    samples_pred_labels_df = pd.DataFrame(samples_pred_labels)
    samples_pred_labels_df = samples_pred_labels_df.apply(lambda x: [GE_taxonomy[i] for i in range(len(x)) if x[i]==1], axis=1)
    #print(samples_pred_proba)
    #return list(samples_pred_labels_df)
    return pd.DataFrame({"Text":text_samples, "Emotions":samples_pred_labels_df[0]})

In [ ]:
predict_samples(["I love the move"], model)

[[7.1276183e-04 9.9928731e-01]]


,Text,Emotions
0,I love the move,label_positive
